# Initializing and functions

In [1]:
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import sif_parser
import scipy
from scipy.signal import savgol_filter
import lmfit
from lmfit import Model
import datetime
import pandas as pd
import csv
import cv2

scl=0.45
#############################################################################
SMALL_SIZE = 30*scl
MEDIUM_SIZE = 33*scl
BIGGER_SIZE = 35*scl
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
# plt.rcParams["font.family"] = "Times New Roman"
# plt.rcParams['text.usetex'] = True
#############################################################################
%matplotlib tk
def logistic(x, a, b, c, d):
        return a / (1. + np.exp(-c * (x - d))) + b
def square(x, a, b, c):
    return a*x**2 + b*x + c

# Functions for resolution evaluation

In [2]:
import numpy as np
from pyhank import qdht, HankelTransform
import matplotlib.pyplot as plt

def smoothAvg(data, box_pts):
    y = data[:,1]
    y_smooth = smoothAvgAux(y, box_pts)
    if box_pts == 0:
        return np.array([data[:,0], y_smooth]).transpose()
    else:
        return np.array([data[:-box_pts,0], y_smooth[:-box_pts]]).transpose()

def smoothAvgAux(y, box_pts):
    if box_pts == 0:
        return y
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def lsf2psf(lsf, x, control):
    r = np.linspace(start=0.0, stop=np.max(np.abs(x)), num=control['nr'])
    lsf_mat = np.tile(np.abs(lsf).transpose(), [len(r), 1])
    X, R = np.meshgrid(x, r)
    dx = np.abs(x[2] - x[1])

    integrand = - 1 / np.pi * lsf_mat * R / ( np.abs(X) * np.sqrt(X**2 - R**2 + 1e-20)) * dx
    integrand = np.nan_to_num(integrand, nan=0) * (np.abs(X) > R)
    integral = np.sum(integrand, axis=1)
    psf = np.abs(np.diff(integral)[control['spare']:-control['spare']])

    psf = smoothAvgAux(psf, control['blurKernelSizePSF'])[control['blurKernelSizePSF']:]
    return psf

def mtf2dqe(psf, mtf, control):
    absorption = 0.08
    g1 = absorption
    g2 = np.sum(np.abs(psf))
    g3 = 0.5
    dqe = g1 / (1 + 1 / (g2 * g3 * np.abs(mtf / 100)**2))
    dqe = 100 * smoothAvgAux(dqe, control['blurKernelSizeDQE'])[control['blurKernelSizeDQE']:]
    return dqe

def psf2mtf(r, psf, control):
    """Compute the MTF of a PSF with cylindrical symmetric using Hankel transform"""
    H = HankelTransform(order=0, radial_grid=r)

    psf_high_res = np.zeros_like(r)
    psf_high_res[:len(psf)] = psf

    ht = np.abs(H.qdht(psf_high_res))
    ht /= (np.max(ht) + 1e-12)
    smoothMTF = smoothAvg(np.array([H.v, ht]).transpose(), control['blurKernelSizeMTF'])
    smoothMTF = smoothMTF[(control['blurKernelSizeMTF'] - 0):, :]
    smoothMTF[:, 0] -= smoothMTF[0, 0]
    smoothMTF[:, 1] /= (smoothMTF[0, 1] + 1e-20)
    smoothMTF[:, 1] *= 100
    return smoothMTF

def compute_image_characteristics(y, control):
    r = control['CCD_pixel_size'] * np.arange(len(y))
    esf = smoothAvgAux(y, control['blurKernelSize'])
    if control['blurKernelSize'] > 0:
        esf = esf[control['blurKernelSize']: -control['blurKernelSize']]
    lsf = np.abs(smoothAvgAux(np.diff(esf), control['blurKernelSize']))

    data = np.array([r,y]).transpose()
    data = smoothAvg(data, control['blurKernelSize'])
    data[:,0] -= data[0,0]

    x = control['CCD_pixel_size'] * (np.arange(len(lsf)) - np.argmax(lsf))
    psf = lsf2psf(lsf, x, control)

    esf_vec = np.array([control['CCD_pixel_size'] * (np.arange(len(esf)) - np.argmax(np.abs(lsf))), esf]).transpose()
    lsf_vec = np.array([control['CCD_pixel_size'] * (np.arange(len(lsf)) - np.argmax(np.abs(lsf))), lsf]).transpose()
    psf_vec = np.array([control['CCD_pixel_size'] * np.arange(len(psf)), psf]).transpose()

    mtf_vec = psf2mtf(psf_vec[:, 0], psf_vec[:, 1], control)
    dqe_vec = np.array([mtf_vec[:-control['blurKernelSizeDQE'], 0], mtf2dqe(psf_vec[:, 1], mtf_vec[:, 1], control)]).transpose()

    return data, esf_vec, lsf_vec, psf_vec, mtf_vec, dqe_vec

def plot_image_characteristics(esf, lsf, psf, mtf, dqe, control):
    fig, ax = plt.subplots(1, 4)
    # Plot the three lines on the same axes
    ax[0].plot(esf[:, 0], esf[:, 1], label='esf')
    ax[1].plot(psf[:, 0], psf[:, 1], label='psf')
    ax[2].plot(mtf[:, 0], mtf[:, 1], label='mtf')
    ax[2].axhline(3, color='r')
    ax[3].plot(dqe[:, 0], dqe[:, 1], label='dqe')
    ax[2].set_yscale('log')
    ax[3].set_yscale('log')
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()
    ax[3].legend()
    plt.show()

def find_closest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    
    return idx

def get_resolution(mtf, threshold=0.1):
    return mtf[find_closest(mtf[:,1], threshold),0]

def mtf2dqe2(mtf, xray_absorption):
    T2 = np.abs(mtf / 100)**2
    g1 = xray_absorption
    
    # Conversion from xray to electrons, similar to all structures
    g2 = 1.

    # For simplicity we assume the all secondary electrons are converted to light 
    g3 = 1.
    
    # The scintillation yield in hybrid structures is fixed
    scintillation_yield = 9 # 1/keV
    electron_energy = 22 # keV
    g4 = electron_energy * scintillation_yield

    dqe = 1 / (1 + (1-g1)/g1 + 1/(g1*g2) + (1-g3)/(g1*g2*g3) + 1/(g1*g2*g3*g4) + (1-T2)/(g1*g2*g3*g4*T2))
    dqe = 100 * dqe
    return dqe

"""
Here start the main module of the script
First, we define the control, which contains some general parameters
Then we extract the image characteristics
"""

control = {
           'CCD_pixel_size': 13e-3,  # in [mm]
           'blurKernelSize': 15,
           'blurKernelSizePSF': 15,
           'blurKernelSizeMTF': 20,
           'blurKernelSizeDQE': 2,
           'spare': 1,
           'nr': 200,
           }

# plt.ion()
# a = 1
# c = 1
# x = np.linspace(-10, 10, 200)
# esf_signal = a / (1 + c * np.exp(-x))
# data, esf, lsf, psf, mtf, dqe = compute_image_characteristics(esf_signal, control)
# plot_image_characteristics(esf, lsf, psf, mtf, dqe, control)
# print()

# Reference signal substraction
## Image is the reference - signal

In [3]:
plt.close()

tck = 'hybrid'
if os.getcwd().split('\\')[-1] == 'Analysis notebook':
    os.chdir('../')
folder = '31-28.10 backround removing/Mask-100um/'
# folder = '31-28.10 backround removing/Different_stopping/'+tck
# folder = '05.12.2023 - Cu source/'
os.chdir(folder+tck)

s = 'sig.asc' # mask-100um is '.asc' and mask-400um is '.txt'
r = 'ref.asc'
with open(s, 'r') as the_file:
    for line in the_file:
        if line.startswith('564.67523'):
            break
    for each_line in the_file:    
        N = np.array([each_line.split(",") for each_line in the_file])
sig = N[:,1:-1]
sig = np.asfarray(sig)
with open(r, 'r') as the_file:
    for line in the_file:
        if line.startswith('564.67523'):
            break
    for each_line in the_file:    
        N = np.array([each_line.split(",") for each_line in the_file])
ref = N[:,1:-1]
ref = np.asfarray(ref)
im = np.true_divide(sig,ref)
# sig[sig > 5*np.std(sig)] = np.std(sig)
image = np.rot90(im, k=-1) #dark side should be to the left, if not change k switch 1 to -1
if tck == '3.6_um':
    image[852,429] = 0.5
plt.imshow(image)#, cmap='gray')
plt.title('Sample: uniform {}um'.format(tck))
plt.savefig(tck+'.png')
plt.show()
os.chdir('../../../')


# Image aligning and slicing
## binary
is a binary mask to difrenciate between the covered and the uncovered area
## rotated
is the binary mask where the line between the cover and uncover is rotated to aligned with y axis
## roi
is the ROI where step function will be fitted to avoid overfitting and picture artefacts

In [8]:
plt.close()
def first_nonzero(arr, axis, invalid_val=-1): #Find first non-zero value in every column of a 2d array
    mask = arr>1e-6
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)
def line(x, slope, intercept):
    return slope*x + intercept
plt.subplot(221)
plt.title('Original image')
plt.imshow(image, cmap='gray')
binary = np.zeros(np.shape(image))
binary[image>2.5*np.std(image)] = 255 # if doesn't work use 1.5 instead of 2.5
binary = scipy.ndimage.median_filter(binary,size=(25,25))
plt.subplot(222)
plt.title('Binary mask')
plt.imshow(binary, cmap='gray')
vec = first_nonzero(binary[:,100:-100], axis=1, invalid_val=-1)
x = np.linspace(0,len(vec)-1,len(vec))
gmodel = Model(line)
result = gmodel.fit(vec, x=x, slope=((vec[-1]-vec[1])/(x[-1]-x[1])), intercept=vec[1])
theta = np.arctan(result.params['slope'].value)*180/np.pi
plt.subplot(223)
plt.title('Binary mask\nalign verticaly')
rotated = scipy.ndimage.rotate(binary, -theta)
plt.imshow(rotated, cmap='gray')
mid = int(np.average(first_nonzero(rotated[50:-50,:], axis=1, invalid_val=-1)))+20# Add 115 to sample 7.5, Add 10 to Hybrid, add 15 to all 400um 
roi = rotated[100:-100,mid-100:mid+100]
plt.subplot(224)
plt.title('Reigion of interest\nof binary mask')
plt.imshow(roi, cmap='gray')
plt.tight_layout()
plt.show()
print(theta)

5.054136562464148


# Fitting 
The images that here are taken where the sample is tilted so that the top is above the focal plane and the bottom of the image is below the focus plane. This assure us that there is a part of the image is in the focus plane. 

To find focus bit I fit the logistic equation to each horizontal line of the image and find the sharpest line hence where the image focus is in its maximum. 
 

In [9]:
plt.close()
watch=False
image2 = scipy.ndimage.rotate(image, -theta)
image2 = image2[100:-100,mid-165:mid+165]
image3 = np.zeros((np.shape(image2)[0], np.shape(image2)[1]-130)) # This is the image where the lines aligned according to the center of the st
n = np.shape(image2)[0]
slope = np.zeros(n)
p2p = np.zeros(n)
pixel = np.linspace(0,n-1,n)
for i in range(n):
    y = image2[i,:]
    x = np.linspace(0,len(y)-1,len(y))
    gmodel = Model(logistic)
    result = gmodel.fit(y, x=x, a=0.5, c = .1, d=100, b=0.2)
    slope[i] = result.params['c'].value
    p2p[i] = result.params['a'].value
    # print('i = {}, c={:.3f}'.format(i,result.params['c'].value,np.std(y)))
    if watch:
#         if n%30==0:
        plt.plot(x, y,alpha=0.5)
        plt.plot(x, result.best_fit, label='best fit')
        plt.show(block=False)
        plt.pause(0.05)
        plt.close()
#         break
    cen = int(result.params['d'].value)
    image3[i,:] = y[cen-100:cen+100]


# Summing over pixels

In [10]:
print(os.getcwd())
plt.subplot(121)
plt.imshow(image3, aspect='auto')
plt.subplot(122)
slices = 20
image4 = np.zeros((slices, np.shape(image3)[1]))
image4_stdev = np.zeros((slices, np.shape(image3)[1]))
for i in range(slices):
    one = int(len(image3[:,i])/slices)*i
    two = int(len(image3[:,i])/slices)*(i+1)
    for j in range(np.shape(image3[one:two])[1]):
        image4[i,j] = sum(image3[one:two,j])
        image4_stdev[i,j] = np.std(image3[one:two,j])
plt.imshow(image4, aspect='auto')
# plt.tight_layout()
# # slope[slope>(np.average(slope)+3.5*np.std(slope))]=np.average(slope)
# plt.savefig(folder + '/roi'+tck+'av_over50.png')
plt.show()

C:\Users\orrbeer\OneDrive - Technion\My files\Shared lab data\Group members DATA\Orr\GitHub\Resolution_analysis


# Fitting over summed figure

In [16]:

def erfunc(x, mFL, a, b, c): # I don't use here erf I used logistics eq. but it is also possible using erf
    return mFL*scipy.special.erf((x-a)/(b*np.sqrt(2)))+c
watch = True
n2 = np.shape(image4)[0]
resolution2 = np.zeros(n2)
slope2 = np.zeros(n2)
slope2std = np.zeros(n2)
p2p2 = np.zeros(n2)
pixel2 = np.linspace(0,n2-1,n2)
watch=False
control = {
               'CCD_pixel_size': 13e-3,  # in [mm]
               'blurKernelSize': 15,
               'blurKernelSizePSF': 1,
               'blurKernelSizeMTF': 20,
               'blurKernelSizeDQE': 2,
               'spare': 1,
               'nr': n2,
               }
for i in range(n2):
    y = image4[i,:]
    x = np.linspace(0,len(y)-1,len(y))
    gmodel = Model(logistic)
#     result = gmodel.fit(y, x=x, mFL=10, a=100, b = 5, c=30)
    gmodel = Model(logistic)
    result = gmodel.fit(y, x=x, a=0.5, c = .1, d=100, b=0.2)
    slope2[i] = abs(result.params['c'].value)
    slope2std[i] =result.params['c'].stderr
    p2p2[i] = abs(result.params['a'].value)
#     print('i = {}, c={:.3f}'.format(i,result.params['c'].value,np.std(y)))
    if watch:
        plt.plot(x, y,alpha=0.5)
        plt.plot(x, result.best_fit, label='best fit')
        plt.show(block=False)
        plt.pause(0.05)
        plt.close()
sq_size = 100
center =int(11*np.shape(image3)[0]/slices)
amp = ref[mid-sq_size:mid+sq_size, center-sq_size:center+sq_size]
resolution = resolution2[np.argmax(slope2)]
plt.figure()
print(('amp; amp_error; slope; slop_error={:.0f}\t{:.0f}\t{:.3f}\t{:.3f}'.format
          (np.average(amp), np.std(amp),slope2[np.argmin(p2p2)],slope2std[np.argmin(p2p2)])))
plt.subplot(211)
plt.title('Amp={:.0f}+/-{:.0f} \nslope={:.3f}+/-{:.3f}'.format
          (resolution, np.average(amp), np.std(amp),slope2[np.argmin(p2p2)],slope2std[np.argmin(p2p2)]))#,np.average(resolution),np.std(resolution)))

plt.plot(pixel2, p2p2, '--or', alpha=0.7)
plt.xlabel('Pixel')
plt.ylabel('Contrast', color='r')
plt.subplot(212)
plt.plot(pixel2, slope2, '--og', alpha=0.7)
plt.xlabel('Pixel')
plt.ylabel('Slope', color='g')
# plt.savefig(folder+'/fitting_params_'+tck+'av_over50.png')
plt.tight_layout()

amp; amp_error; slope; slop_error=149	11	0.423	0.026


# Evaluation of DQE with several assumptions
the assumptions are soon to be published in a paper

In [36]:

def mtf2dqe2(mtf, xray_absorption, sl_thickness, blurKernelSizeDQE):
    T2 = np.abs(mtf / 100)**2
    g1 = xray_absorption
    
    # Conversion from xray to electrons, similar to all structures
    g2 = 1.

    # The secondary electrons that reach the scintillator are converted to light
    electronic_absorption = 10.7 # 1/um
    N = 1001
    depths = np.linspace(0, sl_thickness, N)
    reaching_electrons = np.exp(-electronic_absorption * depths) # The amound of electrons reaching the scintillator from a each depth
    g3 = np.sum(reaching_electrons) * (1/N)
    
    # The scintillation yield in hybrid structures is fixed
    scintillation_yield = 9 # 1/keV
    electron_energy = 22 # keV
    g4 = electron_energy * scintillation_yield

    dqe = 1 / (1 + (1-g1)/g1 + 1/(g1*g2) + (1-g3)/(g1*g2*g3) + 1/(g1*g2*g3*g4) + (1-T2)/(g1*g2*g3*g4*T2))
    dqe = 100 * dqe#savgol_filter(dqe, blurKernelSizeDQE, 1)
    return dqe

In [78]:
plt.close()
y = image4[np.argmax(slope2),:]
y_err = image4_stdev[np.argmax(slope2),:]
y_err = y_err*20
x = np.linspace(0,len(y)-1,len(y))
data = [x,y]
gmodel = Model(logistic)
result = gmodel.fit(y, x=x, a=0.5, c = .1, d=100, b=0.2)

fig = plt.figure(figsize=(11,4))
# slope2[i] = abs(result.params['c'].value)
# slope2std[i] =result.params['c'].stderr
# p2p2[i] = abs(result.params['a'].value)
#     print('i = {}, c={:.3f}'.format(i,result.params['c'].value,np.std(y)))
ax1 = fig.add_subplot(131)
ax1.set_title('Intensity profile')
xmm = x*0.9/1024 # [pixel]*[mm]/[pixel] 
ax1.plot(xmm, result.best_fit,'k--', label='Fit',linewidth=1)
ax1.plot(xmm, y, label='Data')
ax1.fill_between(xmm, result.best_fit-y_err, result.best_fit+y_err, color='gray', alpha=0.3, label='Std')
ax1.set_xlabel('mm')
ax1.set_ylabel('Counts')
ax1.set_xlim([60*0.9/1024,140*0.9/1024])
ax1.legend()

ax2=fig.add_subplot(132)
ax2.set_title('MTF')

control = {
           'CCD_pixel_size': 0.0008789,  # in [mm]
           'blurKernelSize': 1,
           'blurKernelSizePSF': 1,
           'blurKernelSizeMTF': 1,
           'blurKernelSizeDQE': 1,
           'spare': 1,
           'nr': 200,
           }

dy =  abs(np.gradient(y[:-6],x[1]-x[0]))
data, esf, lsf, psf, mtf, dqe = compute_image_characteristics(y, control)
x = mtf[:,0] # The units of mtf[:,0] is lines/pixels. The value 1024/0.9e3 is pixels per um
dx = x[1]-x[0]
norm_mtf = mtf[:,1]/max(mtf[:,1])

d_mtf =  abs(np.gradient(norm_mtf))
dy_mtf =  abs(d_mtf/dy)
err_mtf = abs(dy_mtf*y_err[6:])
plt.plot(x,norm_mtf, alpha=0.7, label='MTF')
# plt.fill_between(x, norm_mtf-err_mtf,norm_mtf+err_mtf, color='b', alpha=0.1)
ax2.set_xlim([0,300])
ax2.set_xlabel('Lines/mm')
ax2.set_ylabel('MTF (Norm.)')


ax3 = fig.add_subplot(133)
ax3.set_title('DQE')
# These parameters are calculated based on thorough analysis that detailed in my PhD thesis
sl_thickness = 0.2 # Sample thickness
x_ray_abs = 0.00136 # Sample X-ray absorption

dqe = mtf2dqe2(mtf[:,1], x_ray_abs,sl_thickness,blurKernelSizeDQE=1)
ax3.plot(x,savgol_filter(dqe,9,1), alpha=0.7)
ax3.set_xlabel('Lines/mm')
ax3.set_ylabel('DQE')
ax3.set_xlim([0,300])
# The units of mtf[:,0] is lines/pixels. The value 1024/0.9e3 is pixels per um
plt.tight_layout()

C:\Users\orrbeer\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in sqrt
C:\Users\orrbeer\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


In [18]:
plt.imshow(ref)
sq_size = 200

plt.plot(mid,center,'ro')
plt.plot([mid-sq_size,mid-sq_size],[center-sq_size,center+sq_size],'r--')
plt.plot([mid+sq_size,mid+sq_size],[center-sq_size,center+sq_size],'r--')
plt.plot([mid-sq_size,mid+sq_size],[center+sq_size,center+sq_size],'r--')
plt.plot([mid-sq_size,mid+sq_size],[center-sq_size,center-sq_size],'r--')
plt.figure()
plt.imshow(ref[center-sq_size:center+sq_size,mid-sq_size:mid+sq_size])
# plt.plot(mid,center,'ro')

plt.show()